# Preprocessing

Rename Masbaisis_indices_280619.xlsx to Masbasis_indices_280619.xlsx  
Rename Masbasis_2606_2019_color_and_othe_indecies  to Masbasis_260619_color_and_othe_indecies  
Rename/manually change the date format of Masbasis 2021 data from YYMMDD to DDMMYY

In [1]:
%%time

import os
import math
from datetime import datetime as dt
import numpy as np
import pandas as pd
from copy import copy

# Dictionaries
import json
from pprint import pprint

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# To display df nicely in loops
from IPython.display import display 
# display(df1.head()) 
# display(df2.head())

# Display rows and columns Pandas
pd.options.display.max_columns = 100
pd.set_option('display.max_rows',100)

Wall time: 1.57 s


In [2]:
# Prints the current working directory
os.getcwd()
# os.listdir()

'C:\\Users\\fahad\\MegaSync\\NMBU\\GitHub\\vPheno'

## Finding Username folder to make general path for multi PC use

In [3]:
username = str(os.getcwd()).split('\\')[2]
username

'fahad'

## ToDo:  
  
2019 2020 data is fine  
Cant use 2017 because of the blue band  
Distt is not normal in 2018 robot  
AREA UNDER THE CURVE in each season for 2018 2017  

## Importing Data

In [4]:
path = r'C:\\Users\\'+username+'\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021'
parent_path = './Data/'
complete_df_path = parent_path+'1. complete_datasets/'

# Create export_path folder if not exists already
os.makedirs(complete_df_path, exist_ok=True)

os.listdir(path)

['2019 Staur Graminor',
 '2019 Vollebekk Graminor',
 '2019 Vollebekk Masbasis',
 '2020 Staur Graminor',
 '2020 Staur Masbasis_bandsNA',
 '2020 Vollebekk Graminor',
 '2020 Vollebekk Masbasis',
 '2020 Vollebekk Robot',
 '2021 Vollebekk Masbasis_yieldNA']

## Data Preparation
### Creating list of all files

In [5]:
# Get the list of all files in directory tree at given path

files_with_address = []
files_list = []

for (dirpath, dirnames, filenames) in os.walk(path):
    files_with_address += [os.path.join(dirpath, file) for file in filenames]
    files_list.extend(filenames)
    
print(len(files_with_address), 'files founnd in the directory')
# files_with_address
# files_list
files_with_address_bkp = copy(files_with_address)

87 files founnd in the directory


## Data Checking/control

### Check for duplicate filenames

In [6]:
print('Total number of files are :', len(files_list))

print('Number of unique file names are:', len(set(files_list)))

print('There is/are', len(files_list) - len(set(files_list)),'duplicate file name/names.')

Total number of files are : 87
Number of unique file names are: 87
There is/are 0 duplicate file name/names.


### Checking if there are multiple sheets in the files

In [7]:
# Print number of sheets in all files
print('The following files have multiple sheets.')

list_multi_sheet = []
for file in files_with_address:
    
    xl_file = pd.ExcelFile(file,engine='openpyxl')
    number_of_sheets = len(xl_file.sheet_names)
    if number_of_sheets > 1:
        print(number_of_sheets, os.path.basename(file), 'in folder', os.path.basename(os.path.dirname(file))
)
        list_multi_sheet.append(file)

The following files have multiple sheets.
3 19BMLFN3 - MASBASIS yield trial Staur 2019.xlsx in folder 2019 Staur Graminor
4 19TvPhenores.xlsx in folder 2019 Vollebekk Graminor
4 19BMLGI1 - MASBASIS yield trial Vollebekk 2019.xlsx in folder 2019 Vollebekk Masbasis
3 Staur-Graminor-Masbasis_2020.xlsx in folder 2020 Staur Graminor
2 20BMLFN3 - MASBASIS avlingsforsøk Staur 2020 lodging data.xlsx in folder 2020 Staur Masbasis_bandsNA
2 Masbasis_2020_staur.xlsx in folder 2020 Staur Masbasis_bandsNA
2 Staur_maturity_heading_yield_2020.xlsx in folder 2020 Staur Masbasis_bandsNA
3 20BMLGI1_2020_tm.xlsx in folder 2020 Vollebekk Masbasis
3 Masbasis_Mica_2020_all_dates_MEDIAN_DP.xlsx in folder 2020 Vollebekk Masbasis
3 ROBOT_2020.xlsx in folder 2020 Vollebekk Robot


### Removing files with multiple sheets from the list

In [8]:
# Removing files with multiple sheets from the list

for i in list_multi_sheet:
    files_with_address.remove(i)
len(files_with_address)

77

### Removing files without dates
(with 2019 in name means they dont have date format)

In [9]:
files_w_2019 = []
for file in files_with_address:
    file_name = os.path.basename(file)
    if '2019' in file_name:
        print(file_name, 'in folder', os.path.basename(os.path.dirname(file)))
        files_w_2019.append(file)
    if '2020' in file_name:
        print(file_name, 'in folder', os.path.basename(os.path.dirname(file)))
        files_w_2019.append(file)

Staur_Graminor_2019_median PLT918-2050 240719 070819 150819 210819 300819.xlsx in folder 2019 Staur Graminor
Staur_Masbasis_2019 PLT101-866 240719 070819 150819 300819.xlsx in folder 2019 Staur Graminor
2020TGraminor-Vollebekk-res.xlsx in folder 2020 Vollebekk Graminor


In [10]:
# Removing from list
for i in files_w_2019:
    print(i)
    files_with_address.remove(i)

C:\\Users\\fahad\MegaSync\NMBU\Master Thesis\Data\Feb2021\2019 Staur Graminor\Staur_Graminor_2019_median PLT918-2050 240719 070819 150819 210819 300819.xlsx
C:\\Users\\fahad\MegaSync\NMBU\Master Thesis\Data\Feb2021\2019 Staur Graminor\Staur_Masbasis_2019 PLT101-866 240719 070819 150819 300819.xlsx
C:\\Users\\fahad\MegaSync\NMBU\Master Thesis\Data\Feb2021\2020 Vollebekk Graminor\2020TGraminor-Vollebekk-res.xlsx


In [11]:
len(files_with_address)
# files_with_address

74

### Checking number of unique cultivars in the field

In [12]:
# plots_data = pd.read_excel(files_with_address[0],engine='openpyxl')
# # Pandas converts 'NA' string to NaN. Need to change those to 
# # some string to get a count as NaNs are not counted as unique values

# plots_data.Name.fillna('-', inplace=True)
# plots_data.CodeName.fillna('-', inplace=True)

# # Creating a new column as multiple plots were named 'NA' but the 
# # CodeName was different for each one of them
# plots_data['NameCode'] = plots_data.Name+plots_data.CodeName

# plots_data
# len(plots_data.NameCode.unique())
# plots_data.NameCode.value_counts()
# # plots_data.NameCode.value_counts().sum()
# # plots_data

## Importing data files to Pandas

In [13]:
%%time

all_df = []
for data in files_with_address:
    file_name = os.path.splitext(os.path.basename(data))[0]

    # Replce all invalid characters in the name
    file_name = file_name.replace("-", "_")
    file_name = file_name.replace(" ", "_")
    file_name = file_name.replace("(", "")
    file_name = file_name.replace(")", "")
    df_name = file_name.replace(".", "")
    # Test: Check if the same date is already present in the current dict key
    if df_name in all_df:
        print(f'A file with the same name {df_name} has already been imported. \n Please check if there is duplication of data.')
        raise NameError
            
    all_df.append(df_name)

    locals()[df_name] = pd.read_excel(data, engine='openpyxl')
    print(df_name, '=====', locals()[df_name].shape)

Staur_070819_mean_median_SP ===== (1328, 6)
Staur_150819_mean_median_SP ===== (1328, 6)
Staur_210819_mean_median_SP ===== (1328, 6)
Staur_240719_mean_median_SP ===== (1328, 6)
Staur_300819_mean_median_SP ===== (1328, 6)
Graminor_250719 ===== (600, 6)
Graminor_050719_plots_826,_837_deleted,_one_missing_row_deleted ===== (597, 6)
Graminor_060619 ===== (600, 6)
Graminor_070819_2 ===== (600, 6)
Graminor_110619 ===== (600, 6)
Graminor_150719 ===== (600, 6)
Graminor_150819 ===== (600, 6)
Graminor_280619_corrected ===== (600, 6)
Graminor_eastwest_020719_NIR_half_missing ===== (600, 6)
Graminor_eastwest_050819 ===== (600, 6)
Graminor_east_110719 ===== (300, 6)
Masbasis_050719_corrected ===== (528, 6)
Masbasis_060619_Indices ===== (528, 6)
Masbasis_070819_correct ===== (528, 6)
Masbasis_150719 ===== (528, 6)
Masbasis_220719_correct ===== (528, 6)
Masbasis_260619_color_and_othe_indecies ===== (528, 6)
Masbasis_290719 ===== (528, 6)
Masbasis_indices_280619 ===== (528, 6)
Staur_090720_M ===== (172

In [14]:
print(f'Total imported {len(all_df)}')

Total imported 74


## Data Control: Check which df have the data column heading we need

### Listing column headings into df

In [15]:
%%time

# Finding max number of columns in all df

len_columns = []
for df in all_df:
    cols_df = locals()[df].columns
    len_columns.append(len(cols_df))
max_cols_in_df = max(len_columns)

# Now creating a empty df to collect all column headings
columns_df = pd.DataFrame(data=range(0,max_cols_in_df), columns = ['ID'])
columns_df.drop('ID', axis=1, inplace=True)

for df in all_df:
    cols_df = locals()[df].columns
    columns_df[df] = pd.Series(cols_df)
columns_df

Wall time: 17.1 ms


,Staur_070819_mean_median_SP,Staur_150819_mean_median_SP,Staur_210819_mean_median_SP,Staur_240719_mean_median_SP,Staur_300819_mean_median_SP,Graminor_250719,"Graminor_050719_plots_826,_837_deleted,_one_missing_row_deleted",Graminor_060619,Graminor_070819_2,Graminor_110619,Graminor_150719,Graminor_150819,Graminor_280619_corrected,Graminor_eastwest_020719_NIR_half_missing,Graminor_eastwest_050819,Graminor_east_110719,Masbasis_050719_corrected,Masbasis_060619_Indices,Masbasis_070819_correct,Masbasis_150719,Masbasis_220719_correct,Masbasis_260619_color_and_othe_indecies,Masbasis_290719,Masbasis_indices_280619,Staur_090720_M,Staur_160720_Mica,Staur_200620_Mica_index,Staur_240720,Staur_250620_Mica_index,Staur_310720,Graminor_eastwest_040720,Graminor_eastwest_040820,Graminor_eastwest_070720_correct,Graminor_eastwest_130720,Graminor_eastwest_140820,Graminor_eastwest_300720,Graminor_east_010720,Graminor_east_170720,Graminor_east_180620,Graminor_east_200720,Graminor_Mica_eastcorrect_west_240620,Masbasis_Mica_010720,Masbasis_Mica_070820,Masbasis_Mica_080720,Masbasis_Mica_120820,Masbasis_Mica_130720,Masbasis_Mica_140820,Masbasis_mica_170720,Masbasis_mica_180620_several_missing_rows_deleted,Masbasis_Mica_220720,Masbasis_mica_240620,Masbasis_Mica_260620,"Masbasis_Mica_300720_duplicate_plots_deleted_1332,1329,1330,1331same_data",Robot_Mica_010720,Robot_Mica_040820,Robot_Mica_070720,Robot_Mica_120820,Robot_Mica_130720,Robot_mica_180620,Robot_Mica_200720,Robot_Mica_220720,Robot_Mica_230620,Robot_Mica_240620,Robot_Mica_250620,Robot_Mica_270720,Robot_Mica_290620,Robot_Mica_300720,Masbasis_mica_020621,Masbasis_mica_090721_excel,Masbasis_mica_170621,Masbasis_mica_220721,Masbasis_mica_230621_excel,Masbasis_mica_240621_excel,Masbasis_mica_260721
0,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID
1,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue,Blue
2,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green
3,Red,Red,Red,Red,Red,Red,Red,NIR,NIR,NIR,Red,Red,NIR,Red,Red,NIR,Red,NIR,Red,NIR,Red,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,Red,NIR,NIR,NIR,NIR,NIR,Red,NIR,NIR,NIR,Red,NIR,Red,NIR,Red,Red,NIR,Red,Red,NIR,Red,NIR,Red,Red,Red,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR,NIR
4,RedEdge,RedEdge,RedEdge,RedEdge,RedEdge,RedEdge,RedEdge,Red,Red,Red,RedEdge,RedEdge,Red,RedEdge,RedEdge,Red,RedEdge,Red,RedEdge,Red,RedEdge,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,RedEdge,Red,Red,Red,Red,Red,RedEdge,Red,Red,Red,RedEdge,Red,RedEdge,Red,RedEdge,RedEdge,Red,RedEdge,RedEdge,Red,RedEdge,Red,RedEdge,RedEdge,RedEdge,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red
5,NIR,NIR,NIR,NIR,NIR,NIR,NIR,RedEdge,RedEdge,RedEdge,NIR,NIR,RedEdge,NIR,NIR,RedEdge,NIR,RedEdge,NIR,RedEdge,NIR,RedEdge,RedEdge,RedEd

### Convert columns_df to dictoionary

In [16]:
%%time

# Function to convert df to dict while dropping nan in each column separately

def comp_dropna(df1):
    return {k: v.dropna().to_dict() for k,v in df1.items()}

columns_dict = comp_dropna(columns_df)
# columns_dict

Wall time: 12 ms


# Standardizing the names

## Creating a dictionary with all dates for a certain field for a certain year

In [17]:
len(all_df)
# all_df

74

In [18]:
%%time

elements_to_strip = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ_-()."

all_df_std = []

field_year_dict = {}

# A reference dict to keep record of the names of files before they got renamed 
renamed_to_from = {}

for df in all_df:
    
    # Getting date from the df name
    date1 = copy(df)

    for x in range(3):
        date1 = date1.rstrip(elements_to_strip)
        date1 = date1.lstrip(elements_to_strip)
        for c in range(3):
            date1 = date1.rstrip(elements_to_strip)
            date1 = date1.lstrip(elements_to_strip)
        date1 = date1.split('_')[0]
    
    field_name = df.split('_')[0]
    field_name = field_name.split('-')[0]

    new_df_name = field_name +'_'+date1
    
    # Drop all columns except the std columns 
    locals()[new_df_name] = locals()[df].copy()
    
    all_df_std.append(new_df_name)
    
    # Creating a dict with all dates for a certain field for a certain year
    dict_key = field_name+'_20'+date1[-2:]
    
    # If the Field name is present in dict then add the date to that key
    # Otherwise, create new key for current field and add the date to it
    if dict_key in field_year_dict:
        # Test: Check if the same date is already present in the current dict key
        if date1 in field_year_dict[dict_key]:
            print(f'Duplicate Data file Error: {date1} is already present in {dict_key}\n Current df {df}\n Conflict with {renamed_to_from[new_df_name]}')
            raise NameError
        field_year_dict[dict_key].append(date1)
    else:
        field_year_dict[dict_key] = [date1]
    
    # Adding new and old names to a dict for record
    renamed_to_from[new_df_name] = df
    print( new_df_name, '*********', df)
#     print(date1)
print('field_year_dict created.')

# pprint(field_year_dict)
# all_df_std


Staur_070819 ********* Staur_070819_mean_median_SP
Staur_150819 ********* Staur_150819_mean_median_SP
Staur_210819 ********* Staur_210819_mean_median_SP
Staur_240719 ********* Staur_240719_mean_median_SP
Staur_300819 ********* Staur_300819_mean_median_SP
Graminor_250719 ********* Graminor_250719
Graminor_050719 ********* Graminor_050719_plots_826,_837_deleted,_one_missing_row_deleted
Graminor_060619 ********* Graminor_060619
Graminor_070819 ********* Graminor_070819_2
Graminor_110619 ********* Graminor_110619
Graminor_150719 ********* Graminor_150719
Graminor_150819 ********* Graminor_150819
Graminor_280619 ********* Graminor_280619_corrected
Graminor_020719 ********* Graminor_eastwest_020719_NIR_half_missing
Graminor_050819 ********* Graminor_eastwest_050819
Graminor_110719 ********* Graminor_east_110719
Masbasis_050719 ********* Masbasis_050719_corrected
Masbasis_060619 ********* Masbasis_060619_Indices
Masbasis_070819 ********* Masbasis_070819_correct
Masbasis_150719 ********* Masba

## Test: Check if there are duplicate datasets/names in all_df_std

In [19]:

if len(all_df_std) > len(set(all_df_std)):
    duplicates = len(all_df_std) - len(set(all_df_std))
    if duplicates>1:
        verb, plural='are', 's'
    else:
        verb, plural='is', ''
    print(f'Error:\nThere {verb} {duplicates} duplicate name{plural} in the datasets out of total {len(all_df_std)}.\n \
    Make sure no dataset has been lost because of data being separated in east/west fields on the same date')
    # Printing the names of the duplicate datasets, if any
    find_duplicates=[]
    for i in all_df_std:
        if i not in find_duplicates:
            find_duplicates.append(i)
        else:
            print(f'Duplicate dataset named \'{i}\',\n')
    raise NameError
    
else:
    print('No duplicate dataset found')

No duplicate dataset found


## Arranging the dates in field_year_dict in ascending order

In [20]:
sorted_field_year_dict = {}
for key, dates_list in field_year_dict.items():
    # Converting the dates to a datetime date object and sorting them in list
    sorted_dated = sorted([
        dt.strptime(date, '%d%m%y').date()
        for date in dates_list])

    sorted_field_year_dict[key] = sorted_dated

pprint(sorted_field_year_dict)

{'Graminor_2019': [datetime.date(2019, 6, 6),
                   datetime.date(2019, 6, 11),
                   datetime.date(2019, 6, 28),
                   datetime.date(2019, 7, 2),
                   datetime.date(2019, 7, 5),
                   datetime.date(2019, 7, 11),
                   datetime.date(2019, 7, 15),
                   datetime.date(2019, 7, 25),
                   datetime.date(2019, 8, 5),
                   datetime.date(2019, 8, 7),
                   datetime.date(2019, 8, 15)],
 'Graminor_2020': [datetime.date(2020, 6, 18),
                   datetime.date(2020, 6, 24),
                   datetime.date(2020, 7, 1),
                   datetime.date(2020, 7, 4),
                   datetime.date(2020, 7, 7),
                   datetime.date(2020, 7, 13),
                   datetime.date(2020, 7, 17),
                   datetime.date(2020, 7, 20),
                   datetime.date(2020, 7, 30),
                   datetime.date(2020, 8, 4),
                   da

## Summary of remaining data

In [21]:
for field, dates in sorted_field_year_dict.items():
    rows_df = []
    for date in dates:
        field_name = field.split('_')[0]+'_'+date.strftime('%d%m%y')
        temp_df = locals()[field_name].copy()
        rows_df.append(temp_df.shape[0])
#         print(field_name, temp_df.shape)
    print(field.split('_')[0], date.year, 'Rows:', rows_df)
# sorted_field_year_dict
#         print(field_name)

Staur 2019 Rows: [1328, 1328, 1328, 1328, 1328]
Graminor 2019 Rows: [600, 600, 600, 600, 597, 300, 600, 600, 600, 600, 600]
Masbasis 2019 Rows: [528, 528, 528, 528, 528, 528, 528, 528]
Staur 2020 Rows: [1722, 1722, 1722, 1722, 1722, 1722]
Graminor 2020 Rows: [400, 757, 400, 800, 800, 800, 400, 400, 787, 793, 800]
Masbasis 2020 Rows: [688, 688, 688, 688, 688, 688, 688, 688, 688, 688, 688, 688]
Robot 2020 Rows: [96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96]
Masbasis 2021 Rows: [696, 696, 696, 696, 696, 696, 696]


# ToDo: Dropping NAN

## Finding NAN values
### ToDo: Test: Raise error if missing values found

In [22]:
# Finding number of missing values in each dataframe
df_with_nan = []
missing_values = False
for df in all_df:
    if locals()[df].isna().sum().sum() > 0:
        print(f'Total missing values in {df} are {locals()[df].isna().sum().sum()}')
        missing_values = True
        df_with_nan.append(df)
#     if len(df_with_nan) > 0:
#         raise ValueError
if not missing_values:
    print('No missing value found in any dataframe')

Total missing values in Staur_210819_mean_median_SP are 269
Total missing values in Graminor_eastwest_020719_NIR_half_missing are 300


In [23]:
df_with_nan

['Staur_210819_mean_median_SP', 'Graminor_eastwest_020719_NIR_half_missing']

In [24]:
# Finding which column has NAN values
for df in df_with_nan:
    print(f'{df}:\n {locals()[df].shape[1]-locals()[df].dropna(axis=1).shape[1]} columns or {locals()[df].shape[0]-locals()[df].dropna().shape[0]} rows to be dropped,')

Staur_210819_mean_median_SP:
 5 columns or 61 rows to be dropped,
Graminor_eastwest_020719_NIR_half_missing:
 1 columns or 300 rows to be dropped,


## ToDo: Automate: Drop rows with missing values in df_with_nan

In [25]:
for df in df_with_nan:
    print(f'{df, locals()[df].shape} Before dropping')
#     locals()[df].dropna(inplace=True)
    print(f'{df, locals()[df].shape} After dropping')


('Staur_210819_mean_median_SP', (1328, 6)) Before dropping
('Staur_210819_mean_median_SP', (1328, 6)) After dropping
('Graminor_eastwest_020719_NIR_half_missing', (600, 6)) Before dropping
('Graminor_eastwest_020719_NIR_half_missing', (600, 6)) After dropping


Since it is only NIR values missing in **Graminor_eastwest_020719_NIR_half_missing**, we can keep them since they will be adjusted in Simpsons integration process. And we can use the data from other indices which is available. Same logic can be applied to the other dataset **Staur_210819_mean_median_SP**.

## ORRR

## ToDo: Droppping df with Nan from the all_df_std

In [26]:
print(f'Number of items in all_df is {len(all_df)}')

Number of items in all_df is 74


In [27]:
# for df in df_with_nan:
#     all_df.remove(df)

###  ToDo: Update field_year_dict and sorted_field_year_dict after dropping the dataset

In [28]:
print(f'Number of items in all_df now is {len(all_df)}')

Number of items in all_df now is 74


## Summary of remaining data

In [29]:
for field, dates in sorted_field_year_dict.items():
    rows_df = []
    for date in dates:
        field_name = field.split('_')[0]+'_'+date.strftime('%d%m%y')
        temp_df = locals()[field_name].copy()
        rows_df.append(temp_df.shape[0])
#         print(field_name, temp_df.shape)
    print(field.split('_')[0], date.year, 'Rows:', rows_df)
# sorted_field_year_dict
#         print(field_name)

Staur 2019 Rows: [1328, 1328, 1328, 1328, 1328]
Graminor 2019 Rows: [600, 600, 600, 600, 597, 300, 600, 600, 600, 600, 600]
Masbasis 2019 Rows: [528, 528, 528, 528, 528, 528, 528, 528]
Staur 2020 Rows: [1722, 1722, 1722, 1722, 1722, 1722]
Graminor 2020 Rows: [400, 757, 400, 800, 800, 800, 400, 400, 787, 793, 800]
Masbasis 2020 Rows: [688, 688, 688, 688, 688, 688, 688, 688, 688, 688, 688, 688]
Robot 2020 Rows: [96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96]
Masbasis 2021 Rows: [696, 696, 696, 696, 696, 696, 696]


# Identify & drop duplicate sub-plots in each datasets, if any

## Checking if duplicates exist

In [30]:
found_duplicates = False
for df in all_df_std:
    duplicates = locals()[df][locals()[df].duplicated(subset='Plot_ID')]['Plot_ID']
    if duplicates.size > 0:
        found_duplicates = True
        print(f'Following {duplicates.size} duplicates in {df} ')
#         for x in range(duplicates.size):
#             print(duplicates.iloc[x])
        ident_dup = locals()[df].shape[0]-locals()[df].drop_duplicates(keep=False).shape[0]
        plot_id_dup = locals()[df].shape[0] - locals()[df].drop_duplicates(subset = 'Plot_ID').shape[0]
        print('Identical Duplicates:', ident_dup)
        print(' Plot_ID  Duplicates:', plot_id_dup)
        print('  Rows to be dropped:', plot_id_dup*2)

# if found_duplicates:
#     raise NameError
# else:
#     print(f'No duplicate subplots found in any dataset.')

Following 1 duplicates in Staur_210819 
Identical Duplicates: 0
 Plot_ID  Duplicates: 1
  Rows to be dropped: 2
Following 108 duplicates in Staur_090720 
Identical Duplicates: 0
 Plot_ID  Duplicates: 108
  Rows to be dropped: 216
Following 108 duplicates in Staur_160720 
Identical Duplicates: 0
 Plot_ID  Duplicates: 108
  Rows to be dropped: 216
Following 108 duplicates in Staur_200620 
Identical Duplicates: 0
 Plot_ID  Duplicates: 108
  Rows to be dropped: 216
Following 108 duplicates in Staur_240720 
Identical Duplicates: 0
 Plot_ID  Duplicates: 108
  Rows to be dropped: 216
Following 108 duplicates in Staur_250620 
Identical Duplicates: 0
 Plot_ID  Duplicates: 108
  Rows to be dropped: 216
Following 108 duplicates in Staur_310720 
Identical Duplicates: 0
 Plot_ID  Duplicates: 108
  Rows to be dropped: 216


## Finding out which plots are duplicate/ non-unique

In [31]:
found_duplicates = False

for df in all_df_std:
    plot_list_series = locals()[df].Plot_ID
    if len(plot_list_series[plot_list_series.duplicated()]) > 0:
        found_duplicates = True
#         print(f'Duplicate subplots in {df} are {plot_list_series[plot_list_series.duplicated()]}')
        print(f'{df} Number of plots {len(plot_list_series)}, vs Number of duplicate plots {len(plot_list_series)-len(set(plot_list_series))}')

# if found_duplicates:
#     raise NameError
# else:
#     print(f'No duplicate subplots found in any dataset.')

Staur_210819 Number of plots 1328, vs Number of duplicate plots 1
Staur_090720 Number of plots 1722, vs Number of duplicate plots 108
Staur_160720 Number of plots 1722, vs Number of duplicate plots 108
Staur_200620 Number of plots 1722, vs Number of duplicate plots 108
Staur_240720 Number of plots 1722, vs Number of duplicate plots 108
Staur_250620 Number of plots 1722, vs Number of duplicate plots 108
Staur_310720 Number of plots 1722, vs Number of duplicate plots 108


## Drop duplicate rows

### Drop only identical duplicates

In [32]:
found_duplicates = False
for df in all_df_std:
    dup_temp = False
    size_before = locals()[df].shape
    # Finding completely identical duplicate entries
    duplicates = locals()[df][locals()[df].duplicated()]['Plot_ID']
    if duplicates.size > 0:
        dup_temp = True
        found_duplicates = True
        # Dropping duplicate entries in place
        locals()[df].drop_duplicates(inplace=True)
        print(f'{duplicates.size} duplicate entries deleted from the {df}.')
    if dup_temp:
        rows_dropped = size_before[0] - locals()[df].shape[0]
        print(size_before[0], locals()[df].shape[0], rows_dropped)
        assert rows_dropped == duplicates.size
if not dup_temp:
    print(f'No **identical** duplicate subplots found/dropped in any dataset.')


No **identical** duplicate subplots found/dropped in any dataset.


In [33]:
### Drop all duplicate entries in Plot_ID

In [34]:
found_duplicates = False
for df in all_df_std:
    dup_temp = False
    size_before = locals()[df].shape
    # Finding identical entries in Plot_ID column
    duplicates = locals()[df][locals()[df].duplicated(subset='Plot_ID')]['Plot_ID']
    if duplicates.size > 0:
        dup_temp = True
        found_duplicates = True
        # Dropping duplicate entries in place
        locals()[df].drop_duplicates(subset = 'Plot_ID', keep=False, inplace=True)
        print(f'{duplicates.size*2} duplicate entries deleted from the {df}.')
    if dup_temp:
        rows_dropped = size_before[0] - locals()[df].shape[0]
#         print(size_before[0], locals()[df].shape[0], rows_dropped)
        assert rows_dropped ==duplicates.size*2
if not dup_temp:
    print(f'{df} No duplicate subplots found/dropped in any dataset.')

2 duplicate entries deleted from the Staur_210819.
216 duplicate entries deleted from the Staur_090720.
216 duplicate entries deleted from the Staur_160720.
216 duplicate entries deleted from the Staur_200620.
216 duplicate entries deleted from the Staur_240720.
216 duplicate entries deleted from the Staur_250620.
216 duplicate entries deleted from the Staur_310720.
Masbasis_260721 No duplicate subplots found/dropped in any dataset.


In [35]:
## Summary of remaining data

In [36]:
for field, dates in sorted_field_year_dict.items():
    rows_df = []
    for date in dates:
        field_name = field.split('_')[0]+'_'+date.strftime('%d%m%y')
        temp_df = locals()[field_name].copy()
        rows_df.append(temp_df.shape[0])
#         print(field_name, temp_df.shape)
    print(field.split('_')[0], date.year, 'Rows:', rows_df)
# sorted_field_year_dict
#         print(field_name)

Staur 2019 Rows: [1328, 1328, 1328, 1326, 1328]
Graminor 2019 Rows: [600, 600, 600, 600, 597, 300, 600, 600, 600, 600, 600]
Masbasis 2019 Rows: [528, 528, 528, 528, 528, 528, 528, 528]
Staur 2020 Rows: [1506, 1506, 1506, 1506, 1506, 1506]
Graminor 2020 Rows: [400, 757, 400, 800, 800, 800, 400, 400, 787, 793, 800]
Masbasis 2020 Rows: [688, 688, 688, 688, 688, 688, 688, 688, 688, 688, 688, 688]
Robot 2020 Rows: [96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96]
Masbasis 2021 Rows: [696, 696, 696, 696, 696, 696, 696]


In [37]:
complete_dataframes = all_df_std.copy()

## Exporting complete datasets

In [38]:
# Exporting complete df
# Defining path
os.makedirs(complete_df_path, exist_ok=True)

# Test: Check if there are duplicate names in the complete_dataframes list
# As the last file with the same name will overwrite the previously exported file, resulting in data loss
if len(complete_dataframes) != len(set(complete_dataframes)):
    print('There are duplicate names in the complete_dataframes list.\nPlease check before exporting as it might result in data loss.')
        
    # Printing the names of the duplicate datasets, if any
    find_duplicates=[]
    for i in complete_dataframes:
        if i not in find_duplicates:
            find_duplicates.append(i)
        else:
            print(f'Duplicate dataset named \'{i}\'')
    raise NameError
    
for df in complete_dataframes:
    locals()[df].to_csv(complete_df_path+df+'.csv', index=False)

## Exporting json

In [24]:
# import json
# a_file = open("Data\std_columns.json", "w")
# json.dump(std_columns, a_file)
# a_file.close()

# # a_file = open("Data\std_columns.json", "r")
# # output = a_file.read()
# # a_file.close()
# # print(output)

# END OF SECTION